### 摘要
---
- 2016年刚刚过去，好奇宝宝想要看看过去一年中各公募基金表现如何，是骡子是马拉出来遛遛
- 从三个角度来看各大基金公司的表现：**规模、收益、风控**
- 从规模上，看总规模排名、权益类规模排名、QDII排名等
- 从收益上，精选2016年各类型前10名基金以及各基金公司在各类型上的综合收益表现
- 从风控上，以2016年12月份中的债市大跌为例，看哪家公司在12月份债市风控做得更好

### 1、规模相关排名
---
- 为了对比的一致性，只选取2016年之前成立的开放式基金（未上市），而且为了更能说明问题，将类型分为货币型、债券型、混合型、股票型
- 参与排名的基金都是在16年9月份发布了最新一期的基金报告的，能够取到基金净值的
- 本部分的排名都是规模相关的排名，包括总规模、权益类规模排名等

In [ ]:
import numpy as np
import pandas as pd

data_info = DataAPI.FundGet(category='E,H,B,M',listStatusCd='UN',operationMode='O',field='ticker,secShortName,category,establishDate,managementFullName,managerName,indexFund,isQdii')
data_info = data_info[data_info['establishDate'] < '20160101']  # 2016年之前成立的基金
data_info = data_info.drop_duplicates(subset=['category','establishDate','managementFullName'])  # 同一只基金A,B,C类去重
fund_assets = DataAPI.FundAssetsGet(reportDate='20160930', field='ticker,netAsset')  # 只有能够取到基金规模的参与统计排名
tmp_asset = pd.merge(data_info, fund_assets, how='inner', on='ticker')

##### 如下是基金公司总规模前5排名，可以看到最高的天弘基金管理规模8297亿，总共109家公募基金管理规模大概在8.2万亿左右。

In [ ]:
tmp_asset.groupby('managementFullName').sum().sort('netAsset', ascending=False)[['netAsset']].head()

,netAsset
managementFullName,
天弘基金管理有限公司,8.296796e+11
华夏基金管理有限公司,5.248992e+11
工银瑞信基金管理有限公司,4.804730e+11
易方达基金管理有限公司,4.090070e+11
汇添富基金管理股份有限公司,3.681766e+11


##### 各个类别的基金规模占比如下，很明显`货币基金规模高达55%，而权益类的不足10%`（其中，B为债券类，E为权益类，H为混合类，M为货币类）

In [ ]:
tmp_asset.groupby('category').sum()[['netAsset']] / tmp_asset.groupby('category').sum()[['netAsset']].sum().head()

,netAsset
category,
B,0.127472
E,0.089062
H,0.234694
M,0.548772


##### 一般来说，总规模并不能体现一家基金公司多么牛逼，更好奇的是主动管理能力
---
- 分两个维度：指数型 + 主动管理型
- **指数型**：跟踪股票指数的所有基金
- **主动管理型**：除掉指数型的其他所有权益型 + 混合型

In [ ]:
# 指数型
tmp_asset[(tmp_asset['category']=='E') & (tmp_asset['indexFund']=='I')].groupby('managementFullName').sum()[['netAsset']].sort('netAsset', ascending=False).head()

,netAsset
managementFullName,
富国基金管理有限公司,6.426651e+10
华夏基金管理有限公司,6.232905e+10
嘉实基金管理有限公司,3.716918e+10
南方基金管理有限公司,3.596887e+10
鹏华基金管理有限公司,3.290575e+10


In [ ]:
### 主动管理型
tmp_asset[(tmp_asset['category'].isin(['E','H'])) & (tmp_asset['indexFund']!='I')].groupby('managementFullName').sum()[['netAsset']].sort('netAsset', ascending=False).head()

,netAsset
managementFullName,
易方达基金管理有限公司,1.394053e+11
嘉实基金管理有限公司,1.176748e+11
华夏基金管理有限公司,1.170146e+11
南方基金管理有限公司,1.129083e+11
招商基金管理有限公司,9.388278e+10


##### 另外一个有趣的点，看看各家基金公司QDII的规模排名，海外投资能力强也是很重要的一部分，如下所示，和权益类规模相似，老司机就是老司机

In [ ]:
tmp_asset[tmp_asset['isQdii'] == 1].groupby('managementFullName').sum()[['netAsset']].sort('netAsset', ascending=False).head()

,netAsset
managementFullName,
易方达基金管理有限公司,1.239331e+10
嘉实基金管理有限公司,1.072452e+10
华夏基金管理有限公司,1.010615e+10
南方基金管理有限公司,8.630241e+09
银华基金管理股份有限公司,7.061862e+09


##### 总的来说，在权益和主动管理这一块，华夏和南方都是佼佼者。

### 2、收益相关排名
---
- 和一般机构处理不一致，这里的排名更精细化，首先`货币基金不参与排名`
- 另外，由于QDII的基金多半是跟踪指数，对投资技能的体现不够强，而且有些基金公司压根没有发QDII基金，为了对比一致性`QDII基金也不参与排名`
- 首要的任务是精选股票型、混合型和债券型的前10名基金，然后排出各类别下综合收益前5名的基金公司
- 综合收益计算按照管理规模进行加权，另外，`在计算股票型基金时，剔除掉了指数型基金`，原因类似，我们的排名体现的是主动管理的能力
- 最后，在研究过程中发现了有些基金复权净值有数据问题（4%左右），故将这些基金也剔除掉了。。。

In [ ]:
fund_16 = DataAPI.FundNavGet(dataDate=u"20161230", field='ticker,endDate,ADJUST_NAV')  
fund_15 = DataAPI.FundNavGet(dataDate=u"20151231", field='ticker,endDate,ADJUST_NAV')
fund_return = pd.merge(fund_15, fund_16, how='inner', on='ticker')
fund_return['return'] = fund_return['ADJUST_NAV_y'] / fund_return['ADJUST_NAV_x'] - 1

# 只取满足条件的基金
tmp_return = pd.merge(data_info[(data_info['isQdii']==0) & (data_info['indexFund']!='I')], fund_return[['ticker','return']], how='inner', on='ticker')

# 复权净值有问题的基金剔除(1 min)
bug_tickers = []
for i in tmp_return['ticker'].tolist():
    test = DataAPI.FundNavGet(ticker=i, beginDate='20151231', endDate='20161230')
    if abs(test['ADJUST_NAV'].pct_change().max()) > 0.1:
        bug_tickers.append(i)
        print 'bug    ' + i)
        
tmp_return = tmp_return[~tmp_return['ticker'].isin(bug_tickers)]

##### 首先来看个整体情况，16年股票型基金平均-10.6%（同期上证综指-12.3%），混合型-9.2%，债券型-1.1%

In [ ]:
tmp_return.groupby('category').mean()

,isQdii,return
category,,
B,0,-0.010675
E,0,-0.106249
H,0,-0.091697


##### A、重点来了，首先是股票类年度10强基金，如下所示，最好收益为17.96%

In [ ]:
tmp_return[tmp_return['category']=='E'].sort('return', ascending=False).head(10)

,ticker,secShortName,category,establishDate,managementFullName,managerName,indexFund,isQdii,return
711,001736,圆信永丰优加生活股票,E,2015-10-28,圆信永丰基金管理有限公司,洪流、范妍,NaN,0,0.179612
670,001616,嘉实环保低碳股票,E,2015-12-30,嘉实基金管理有限公司,李化松,NaN,0,0.165000
703,001714,工银瑞信文体产业股票,E,2015-12-30,工银瑞信基金管理有限公司,胡文彪、袁芳,NaN,0,0.157157
780,001938,中欧时代先锋股票发起式,E,2015-11-03,中欧基金管理有限公司,周应波,NaN,0,0.150282
700,001705,泓德战略转型股票,E,2015-11-10,泓德基金管理有限公司,郭堃,NaN,0,0.124138
637,001528,诺安先进制造股票,E,2015-08-21,诺安基金管理有限公司,韩冬燕,NaN,0,0.114398
200,000577,安信价值精选股票,E,2014-04-21,安信基金管理有限责任公司,陈一峰,NaN,0,0.084877
440,001178,前海开源再融资主题精选股票,E,2015-05-18,前海开源基金管理有限公司,邱杰,NaN,0,0.080504
977,110022,易方达消费行业股票,E,2010-08-20,易方达基金管理有限公司,萧楠,NaN,0,0.072079
337,000974,安信消费医药主题股票,E,2015-03-19,安信基金管理有限责任公司,陈一峰,NaN,0,0.071770


##### B、混合型基金年度10强，最高的高达29.8%

In [ ]:
tmp_return[tmp_return['category']=='H'].sort('return', ascending=False).head(10)

,ticker,secShortName,category,establishDate,managementFullName,managerName,indexFund,isQdii,return
505,001272,兴业聚利灵活配置混合,H,2015-05-07,兴业基金管理有限公司,腊博,NaN,0,0.298520
1435,420003,天弘永定价值成长混合,H,2008-12-02,天弘基金管理有限公司,肖志刚,NaN,0,0.174559
1277,240022,华宝兴业资源优选混合,H,2012-08-21,华宝兴业基金管理有限公司,蔡目荣,NaN,0,0.163399
525,001302,前海金银珠宝主题精选灵活配置混合-A,H,2015-07-09,前海开源基金管理有限公司,谢屹,NaN,0,0.158635
979,110025,易方达资源行业混合,H,2011-08-16,易方达基金管理有限公司,王超,NaN,0,0.136256
623,001491,国泰生益灵活配置混合-A,H,2015-06-19,国泰基金管理有限公司,樊利安,NaN,0,0.133566
793,001985,富国低碳新经济混合,H,2015-12-18,富国基金管理有限公司,杨栋,NaN,0,0.130478
649,001547,兴业聚惠灵活配置混合-A,H,2015-07-08,兴业基金管理有限公司,腊博,NaN,0,0.126459
718,001761,广发安宏回报灵活配置混合-A,H,2015-12-30,广发基金管理有限公司,张芊、王予柯,NaN,0,0.126000
402,001115,广发聚安混合-A,H,2015-03-25,广发基金管理有限公司,谭昌杰,NaN,0,0.123797


##### C、债券型年度10强，最高的为11.35%

In [ ]:
tmp_return[tmp_return['category']=='B'].sort('return', ascending=False).head(10)

,ticker,secShortName,category,establishDate,managementFullName,managerName,indexFund,isQdii,return
754,001872,嘉实丰益信用定期开放债券-C,B,2013-08-21,嘉实基金管理有限公司,刘宁,NaN,0,0.113527
1548,519191,万家城市建设主题纯债债券,B,2014-01-24,万家基金管理有限公司,NaN,NaN,0,0.090491
850,002279,浙商惠盈纯债债券,B,2015-12-23,浙商基金管理有限公司,吕文晔,NaN,0,0.073200
1425,400030,东方添益债券,B,2014-12-15,东方基金管理有限责任公司,徐昀君、吴萍萍,NaN,0,0.072682
1266,240003,华宝兴业宝康债券,B,2003-07-15,华宝兴业基金管理有限公司,李栋梁,NaN,0,0.069303
739,001819,兴全稳益债券,B,2015-09-10,兴全基金管理有限公司,钟明,NaN,0,0.066343
100,000277,博时双月薪定期支付债券,B,2013-10-22,博时基金管理有限公司,陈凯杨,NaN,0,0.065334
859,020019,国泰双利债券-A,B,2009-03-11,国泰基金管理有限公司,吴晨,NaN,0,0.060238
1569,519676,银河强化收益债券,B,2014-06-09,银河基金管理有限公司,杨鑫,NaN,0,0.060106
1069,161716,招商双债增强债券,B,2013-03-01,招商基金管理有限公司,刘万锋,NaN,0,0.060039


##### **下面就要对基金公司进行排名，根据各公司旗下基金来综合排出各类型下的前5名**

##### A、股票型收益排名前5基金公司，需要说明几点
---
- 仅仅按照综合收益来排名的话，前几名都是不怎么知名小基金公司，而且刚巧就一只产品，不具备说服性
- 规模也应该是排名的考量因素之一，这里列示的是比较简单的做法，只对管理超过5只基金的公司进行排名，或者对规模超过某一规模的公司进行排名
- 更一般的，可以对收益和规模设置不同权重来计算各自得分，最后计算总分来排名
- 综合前面的结论，`华夏基金`兼具规模和收益优势，在权益类方面可以称之为老大哥。

In [ ]:
tmp_company = data_info.drop(['establishDate', 'indexFund', 'isQdii'], axis=1).merge(fund_assets, on='ticker', how='inner').merge(fund_return[['ticker', 'return']], on='ticker', how='inner')

equity_rank = pd.DataFrame(tmp_company[tmp_company['category']=='E'].groupby('managementFullName').apply(lambda x: sum(x['netAsset']/x['netAsset'].sum() * x['return']))).rename(columns={0:'综合收益'})
equity_rank['管理资产'] = tmp_company[tmp_company['category']=='E'].groupby('managementFullName').sum()['netAsset']
equity_rank['基金个数'] = tmp_company[tmp_company['category']=='E'].groupby('managementFullName').count()['ticker']
# equity_rank[equity_rank['基金个数'] >= 10].sort('综合收益', ascending=False).head()
equity_rank[equity_rank['管理资产'] >= 1e10].sort('综合收益', ascending=False).head()

,综合收益,管理资产,基金个数
managementFullName,,,
易方达基金管理有限公司,-0.040128,4.221494e+10,24
华夏基金管理有限公司,-0.060338,7.447665e+10,15
银华基金管理股份有限公司,-0.067604,1.913412e+10,11
华安基金管理有限公司,-0.075060,3.164761e+10,20
嘉实基金管理有限公司,-0.077561,6.351609e+10,26


##### B、混合型收益排名前5基金公司，`招商和国泰`在16年混合型有不错表现

In [ ]:
hunhe_rank = pd.DataFrame(tmp_company[tmp_company['category']=='H'].groupby('managementFullName').apply(lambda x: sum(x['netAsset']/x['netAsset'].sum() * x['return']))).rename(columns={0:'综合收益'})
hunhe_rank['管理资产'] = tmp_company[tmp_company['category']=='H'].groupby('managementFullName').sum()['netAsset']
hunhe_rank['基金个数'] = tmp_company[tmp_company['category']=='H'].groupby('managementFullName').count()['ticker']
# hunhe_rank[hunhe_rank['基金个数'] >= 20].sort('综合收益', ascending=False).head()
hunhe_rank[hunhe_rank['管理资产'] >= 3e10].sort('综合收益', ascending=False).head()

,综合收益,管理资产,基金个数
managementFullName,,,
招商基金管理有限公司,0.025952,7.639787e+10,23
国泰基金管理有限公司,-0.006958,3.015412e+10,29
南方基金管理有限公司,-0.016840,1.014946e+11,33
诺安基金管理有限公司,-0.027886,3.381124e+10,17
兴全基金管理有限公司,-0.044746,3.285238e+10,9


##### C、债券型收益排名前5基金公司，综合考虑，`博时、银华、鹏华、国泰`均有不错表现

In [ ]:
bond_rank = pd.DataFrame(tmp_company[tmp_company['category']=='B'].groupby('managementFullName').apply(lambda x: sum(x['netAsset']/x['netAsset'].sum() * x['return']))).rename(columns={0:'综合收益'})
bond_rank['管理资产'] = tmp_company[tmp_company['category']=='B'].groupby('managementFullName').sum()['netAsset']
bond_rank['基金个数'] = tmp_company[tmp_company['category']=='B'].groupby('managementFullName').count()['ticker']
bond_rank[bond_rank['基金个数'] >= 10].sort('综合收益', ascending=False).head()
# bond_rank[bond_rank['管理资产'] >= 2e10].sort('综合收益', ascending=False).head()

,综合收益,管理资产,基金个数
managementFullName,,,
国泰基金管理有限公司,0.041833,4.755755e+09,10
银华基金管理股份有限公司,0.019219,2.002277e+10,11
博时基金管理有限公司,0.017596,5.392315e+10,28
鹏华基金管理有限公司,0.017557,2.724948e+10,18
中银基金管理有限公司,0.017132,3.882898e+10,10


### 3、风控相关排名
---
- 传统风控都从最大回撤，波动率等角度来衡量，但是鉴于今年行情的特殊性，仅对债券型基金进行风控分析
- 从20161128日开始至年底结束，债市经历了一波大调整，可能很多债券基金经理在前期经过高杠杆获得了非常丰厚的收益，但是在这一波调整中也死的最惨
- 通过统计从20161128~20161230期间，债券型基金最大回撤的排名来综看基金经理和基金公司的风控能力

In [ ]:
fund_1230 = DataAPI.FundNavGet(dataDate=u"20161230", field='ticker,endDate,ADJUST_NAV')  
fund_1128 = DataAPI.FundNavGet(dataDate=u"20161128", field='ticker,endDate,ADJUST_NAV')
fund_return1 = pd.merge(fund_1128, fund_1230, how='inner', on='ticker')
fund_return1['return'] = fund_return1['ADJUST_NAV_y'] / fund_return1['ADJUST_NAV_x'] - 1
tmp_risk = pd.merge(data_info[(data_info['category']=='B') & (data_info['isQdii']==0) & (data_info['indexFund']!='I')], fund_return1[['ticker','return']], how='inner', on='ticker').drop(['establishDate','indexFund','isQdii'], axis=1).merge(fund_assets, how='inner', on='ticker')
tmp_risk.mean()

return     -2.144551e-02
netAsset    1.633695e+09
dtype: float64

##### 如上可以看到，在调整期，债券型基金的平均收益是-2.14%，全年是-1.1%，说明主要亏损就发生在这一个月中，接下来分公司来看风控水平，处理方法和前面一致

In [ ]:
risk_manage = pd.DataFrame(tmp_risk.groupby('managementFullName').apply(lambda x: sum(x['netAsset']/x['netAsset'].sum() * x['return']))).rename(columns={0:'综合回撤水平'})
risk_manage['管理资产'] = tmp_risk.groupby('managementFullName').sum()['netAsset']
risk_manage['基金个数'] = tmp_risk.groupby('managementFullName').count()['ticker']
risk_manage[risk_manage['管理资产'] > 2e10].sort('综合回撤水平', ascending=False).head()

,综合回撤水平,管理资产,基金个数
managementFullName,,,
银华基金管理股份有限公司,-0.010187,2.064806e+10,13
招商基金管理有限公司,-0.010539,2.795584e+10,13
嘉实基金管理有限公司,-0.015931,2.519490e+10,14
博时基金管理有限公司,-0.016219,5.780588e+10,37
工银瑞信基金管理有限公司,-0.017179,1.234908e+11,14


##### 综合前面关于债券型基金公司的表现分析，`银华、博时、招商`在债券型基金方面应该算是老司机，兼具规模、收益和风控三方面的优异表现。

### 4、总结
---
- 接下来若是要购买基金，可以适当结合考虑下本文的分析结论，不同基金公司有可能侧重点和擅长点不一样
- 当然，本文还可以继续深入下去看，从基金管理人的角度去入手（MOM的思想），比如博时基金擅长权益类投资，更深入一步我们可以对比博时基金众多基金经理的投资表现和擅长方面，这样会更细致更科学一些。